In [1]:
import bo_methods_lib
# from bo_methods_lib.CS2_bo_functions_multi_dim import LHS_Design, test_train_split
from bo_methods_lib.CS1_create_data import create_y_data, calc_y_exp, create_sse_data
from bo_methods_lib.bo_functions_generic import clean_1D_arrays, LHS_Design, test_train_split
from scipy.stats import qmc
import numpy as np
import pandas as pd
import os

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#LHS Design in python - May switch code to use this instead for conciseness
# emulator = True
emulator = False
seed = 9
LHS_Points =300
d = 2
same_LHS_pts = 5

sampler = qmc.LatinHypercube(d=2, seed = seed)
sample = sampler.random(n=LHS_Points - same_LHS_pts)
l_bounds =  [-2, -2]
u_bounds =  [2, 2]
LHS = qmc.scale(sample, l_bounds, u_bounds)
# print(LHS)

sampler_xtra = qmc.LatinHypercube(d=d, seed = seed + 1)
sample_xtra = sampler_xtra.random(n=same_LHS_pts)
LHS_xtra = qmc.scale(sample_xtra, l_bounds, u_bounds)
LHS_xtra = np.round(LHS_xtra, 6)

LHS = np.vstack((LHS, LHS_xtra))
print(LHS.shape)

Bound_Cut = False
denseX = False

if Bound_Cut == True:
    cut_bounds = '_cut_bounds'
else:
    cut_bounds = ""
    
if denseX == True:
    dense = "_dense"
else:
    dense = ""

(300, 2)


In [3]:
#Creates Experimental data and saves to a csv
Theta_True = np.array([1,-1])
Xexp = np.array([-2,-1,0,1,2]).reshape(-1,1)
n = len(Xexp)
exp_d = 1
noise_std = 0.1**2
d = 2
Yexp = calc_y_exp(Theta_True, Xexp, noise_std)

Exp_data = np.column_stack((Xexp,Yexp))

df = pd.DataFrame(Exp_data, columns = ["Xexp","Yexp"])

path = '../Input_CSVs/Exp_Data/d='+str(exp_d)
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(os.path.join(path,'n='+str(n)+'.csv'))

In [4]:
#Array for Standard Approach
all_p_standard = pd.DataFrame(LHS, columns = ["theta1", "theta2"])

print(Xexp,Xexp.shape)
#Array for Emulator approach
df = all_p_standard.iloc[np.repeat(np.arange(len(all_p_standard)), n)]
df = df.reset_index(drop = True)
# Xexp_repeat = np.repeat(Xexp, LHS_Points, axis = 1).reshape(Xexp.shape[1],-1)
# print(Xexp_repeat.reshape(300,2))
Xexp_repeat = np.tile(Xexp, (len(all_p_standard),1))
#Add Xexp 
for i in range(Xexp.shape[1]):
    df['X'+str(i+1)+'exp'] = Xexp_repeat[:,i]    
all_p_emulator = df

# print(all_p_emulator.shape)
# print(all_p_standard)
# print(all_p_emulator)

[[-2]
 [-1]
 [ 0]
 [ 1]
 [ 2]] (5, 1)


In [5]:
#Create y or SSE data
#Save data
if emulator == False:
    all_2_data = all_p_standard.copy()
    print(all_2_data.shape)
    all_2_in_y = create_sse_data(d,np.array(all_2_data),Xexp,Yexp)
    all_2_data['Yexp'] = all_2_in_y
#     
    all_2_data_ln_obj = all_p_standard.copy()
    print(all_2_data.shape)
    all_2_in_y_ln_obj = create_sse_data(d,np.array(all_2_data_ln_obj),Xexp,Yexp, obj = "LN_obj")
    all_2_data_ln_obj['Yexp'] = all_2_in_y_ln_obj
    
    titles_all = np.array(['all_st_data','all_st_ln_obj_data'])
    len_data = str(len(all_2_data))
    all_array = [all_2_data,all_2_data_ln_obj]

else:
    all_3_data = all_p_emulator.copy()
    all_3_in_y = create_y_data(all_3_data)
    all_3_data['Yexp'] = all_3_in_y
    
    titles_all = np.array(['all_emul_data'])
    len_data = str(len(all_3_data))
    all_array = [all_3_data]
    
# print(all_array)

# Save Train Data/Test Data
for i in range(len(titles_all)):
    path = 'Input_CSVs/Train_Data/d=' + str(d) +"/"+ titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    all_array[i].to_csv(os.path.join(path,'t='+len_data + cut_bounds+dense)+ '.csv')

(300, 2)
(300, 3)
